In [1]:
from __future__ import division
import cv2
from mtcnn.mtcnn import MTCNN
from eye import Eye
from calibration import Calibration
import dlib
import os
from keras.models import load_model

Using TensorFlow backend.


In [31]:
detector = MTCNN()

In [32]:

calibration = Calibration()

#_face_detector = dlib.get_frontal_face_detector()

_predictor = dlib.shape_predictor("trained_models/shape_predictor_68_face_landmarks.dat")

In [33]:
def pupils_located(eye_left,eye_right):
    try:
        int(eye_left.pupil.x)
        int(eye_left.pupil.y)
        int(eye_right.pupil.x)
        int(eye_right.pupil.y)
        return True
    except Exception:
        return False

def pupil_left_coords(eye_left,eye_right):
    if pupils_located(eye_left,eye_right):
        x = eye_left.origin[0] + eye_left.pupil.x
        y = eye_left.origin[1] + eye_left.pupil.y
        return (x, y)

def pupil_right_coords(eye_left,eye_right):
    if pupils_located(eye_left,eye_right):
        x = eye_right.origin[0] + eye_right.pupil.x
        y = eye_right.origin[1] + eye_right.pupil.y
        return (x, y)

def annotated_frame(frame_g,eye_left,eye_right):
    if pupils_located(eye_left,eye_right):
        color = (0, 255, 0)
        x_left, y_left = pupil_left_coords(eye_left,eye_right)
        x_right, y_right = pupil_right_coords(eye_left,eye_right)
        cv2.line(frame, (x_left - 5, y_left), (x_left + 5, y_left), color)
        cv2.line(frame, (x_left, y_left - 5), (x_left, y_left + 5), color)
        cv2.line(frame, (x_right - 5, y_right), (x_right + 5, y_right), color)
        cv2.line(frame, (x_right, y_right - 5), (x_right, y_right + 5), color)

    return frame



def horizontal_ratio(eye_left,eye_right):
      
    if pupils_located(eye_left,eye_right):
        pupil_left = eye_left.pupil.x / (eye_left.center[0] * 2 - 10)
        pupil_right = eye_right.pupil.x / (eye_right.center[0] * 2 - 10)
        return (pupil_left + pupil_right) / 2

def vertical_ratio(eye_left,eye_right):

    if pupils_located(eye_left,eye_right):
        pupil_left = eye_left.pupil.y / (eye_left.center[1] * 2 - 10)
        pupil_right = eye_right.pupil.y / (eye_right.center[1] * 2 - 10)
        return (pupil_left + pupil_right) / 2

def is_right(eye_left,eye_right):
    if pupils_located(eye_left,eye_right):
        return horizontal_ratio(eye_left,eye_right) <= 0.35

def is_left(eye_left,eye_right):
    if pupils_located(eye_left,eye_right):
        return horizontal_ratio(eye_left,eye_right) >= 0.65

def is_center(eye_left,eye_right):
    if pupils_located(eye_left,eye_right):
        return is_right(eye_left,eye_right) is not True and is_left(eye_left,eye_right) is not True

In [25]:
cam = cv2.VideoCapture(0)
while True:
    ret,frame = cam.read()
    key_pressed = cv2.waitKey(1)&0xFF
    if ret==False:
        print("Something went wrong")
        continue
    if key_pressed == ord('q'):
        break
    frame_g = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = detector.detect_faces(frame)
    for i in range(len(faces)) :
        
        #eye gaze --------------------------------------------------------
        
        eye_left = None
        eye_right = None
        bounding_box = faces[i]['box']
        x1, y1, width, height = bounding_box
#         face = [(x1,y1),(x1+width,y1+height)]
        face = dlib.rectangle(x1,y1,x1+width,y1+height)
        try:
            landmarks = _predictor(frame, face)
            

        except IndexError:
            eye_left = None
            eye_right = None
            
        frame = annotated_frame(frame_g,eye_left,eye_right)
        
        text=""
        if is_right(eye_left,eye_right):
            text = "Looking right"
        elif is_left(eye_left,eye_right):
            text = "Looking left"
        elif is_center(eye_left,eye_right):
            text = "Looking center"
        
        
        cv2.putText(frame, text, (90, 60), cv2.FONT_HERSHEY_DUPLEX, 1.6, (147, 58, 31), 2)

        left_pupil = pupil_left_coords(eye_left,eye_right)
        right_pupil = pupil_right_coords(eye_left,eye_right)
        cv2.putText(frame, "Left pupil:  " + str(left_pupil), (90, 130), cv2.FONT_HERSHEY_DUPLEX, 0.9, (147, 58, 31), 1)
        cv2.putText(frame, "Right pupil: " + str(right_pupil), (90, 165), cv2.FONT_HERSHEY_DUPLEX, 0.9, (147, 58, 31), 1)
        
        
        cv2.rectangle(frame,(bounding_box[0], bounding_box[1]),(bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),(0,155,255),2)
    
    
    
    cv2.imshow("faces",frame)
    


cam.release()
cv2.destroyAllWindows()

In [ ]:
cam = cv2.VideoCapture(0)
while True:
    ret,frame = cam.read()
    key_pressed = cv2.waitKey(1)&0xFF
    if ret==False:
        print("Something went wrong")
        continue
    if key_pressed == ord('q'):
        break
    frame_g = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = detector.detect_faces(frame)
    for i in range(len(faces)) :
        
        #eye gaze --------------------------------------------------------
        
        eye_left = None
        eye_right = None
        bounding_box = faces[i]['box']
        x1, y1, width, height = bounding_box
#         face = [(x1,y1),(x1+width,y1+height)]
        face = dlib.rectangle(x1,y1,x1+width,y1+height)
        try:
            landmarks = _predictor(frame, face)
            eye_left = Eye(frame_g, landmarks, 0, calibration)
            eye_right = Eye(frame_g, landmarks, 1, calibration)

        except IndexError:
            eye_left = None
            eye_right = None
            
        frame = annotated_frame(frame_g,eye_left,eye_right)
        
        text=""
        if is_right(eye_left,eye_right):
            text = "Looking right"
        elif is_left(eye_left,eye_right):
            text = "Looking left"
        elif is_center(eye_left,eye_right):
            text = "Looking center"
        
        
        cv2.putText(frame, text, (90, 60), cv2.FONT_HERSHEY_DUPLEX, 1.6, (147, 58, 31), 2)

        left_pupil = pupil_left_coords(eye_left,eye_right)
        right_pupil = pupil_right_coords(eye_left,eye_right)
        cv2.putText(frame, "Left pupil:  " + str(left_pupil), (90, 130), cv2.FONT_HERSHEY_DUPLEX, 0.9, (147, 58, 31), 1)
        cv2.putText(frame, "Right pupil: " + str(right_pupil), (90, 165), cv2.FONT_HERSHEY_DUPLEX, 0.9, (147, 58, 31), 1)
        
        
        cv2.rectangle(frame,(bounding_box[0], bounding_box[1]),(bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),(0,155,255),2)
    
    
    
    cv2.imshow("faces",frame)
